In [70]:
from sklearn.tree import DecisionTreeClassifier, export_graphviz
import pandas as pd
import numpy as np
from time import time
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import SGDRegressor
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

from sklearn.model_selection import cross_validate
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score , classification_report, mean_squared_error, r2_score
from sklearn.metrics import precision_score, recall_score, accuracy_score, classification_report
from sklearn.model_selection import learning_curve, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import *
from sklearn import metrics
from sklearn.model_selection import train_test_split

In [71]:

# Let's first load the data and take a look at what we have.
df = pd.read_csv('Heathrow_combined.csv')

In [72]:
df=df.dropna()

In [73]:
df

,Date,Nox_tropo,Nox_ground,tavg,tmin,tmax,prcp,wdir,wspd,pres
0,10/7/18,232.1240,59.746988,18.7,15.5,22.6,0.0,75.0,12.5,1023.9
1,11/7/18,168.0445,73.870523,18.6,13.9,23.4,0.0,45.0,10.1,1021.8
2,12/7/18,194.0030,59.394005,19.2,13.7,24.4,0.0,52.0,8.4,1021.5
3,13/7/18,343.2730,68.192323,20.6,15.7,26.8,0.0,135.0,10.1,1021.8
4,14/7/18,190.1570,78.645600,21.8,14.9,27.8,0.0,177.0,10.2,1020.0
...,...,...,...,...,...,...,...,...,...,...
772,6/1/21,85.2440,40.983786,3.2,0.1,5.6,0.5,89.0,11.3,1019.5
773,7/1/21,163.9400,37.200143,-0.1,-1.7,2.1,0.0,277.0,10.6,1018.6
774,9/1/21,282.0585,58.818259,0.1,-2.1,2.6,0.0,178.0,6.6,1026.4
775,16/1/21,147.2020,37.496792,4.8,-0.8,8.3,0.5,214.0,14.8,1020.0


In [74]:
df.drop(['Date'], 
               axis=1,
              inplace=True)


In [75]:
from sklearn.model_selection import train_test_split

In [76]:
# Separate Target Variable and Predictor Variables
TargetVariable=['Nox_tropo']
Predictors=['Nox_ground', 'tavg', 'tmin', 'tmax', 'prcp', 'wdir', 'wspd','pres']

X=df[Predictors].values
y=df[TargetVariable].values
### Sandardization of data ###
from sklearn.preprocessing import StandardScaler
PredictorScaler=StandardScaler()
TargetVarScaler=StandardScaler()
 
# Storing the fit object for later reference
PredictorScalerFit=PredictorScaler.fit(X)
TargetVarScalerFit=TargetVarScaler.fit(y)
 
# Generating the standardized values of X and y
X=PredictorScalerFit.transform(X)
y=TargetVarScalerFit.transform(y)
 
# Split the data into training and testing set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [77]:
from sklearn.metrics import make_scorer
scorer = make_scorer(mean_squared_error, greater_is_better=False)

In [78]:
from sklearn.svm import SVR

In [79]:
# Tuning of parameters for regression by cross-validation
K = 5               # Number of cross valiations

# Parameters for tuning
parameters = [{'kernel': ['rbf'], 'gamma': [1e-4, 1e-3, 0.01, 0.1, 0.2, 0.5, 0.6, 0.9],'C': [1, 10, 100, 1000, 10000]}]
print("Tuning hyper-parameters")
svr = GridSearchCV(SVR(epsilon = 0.01), parameters, cv = K,scoring=scorer)
svr.fit(X, y)

# Checking the score for all parameters
print("Grid scores on training set:")
means = svr.cv_results_['mean_test_score']
stds = svr.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, svr.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"% (mean, std * 2, params))

Tuning hyper-parameters


/Users/hridoy/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/Users/hridoy/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/Users/hridoy/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/Users/hridoy/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for 

/Users/hridoy/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/Users/hridoy/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/Users/hridoy/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/Users/hridoy/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for 

/Users/hridoy/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/Users/hridoy/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/Users/hridoy/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/Users/hridoy/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for 

/Users/hridoy/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/Users/hridoy/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/Users/hridoy/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/Users/hridoy/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for 

/Users/hridoy/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/Users/hridoy/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/Users/hridoy/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/Users/hridoy/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for 

/Users/hridoy/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/Users/hridoy/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/Users/hridoy/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/Users/hridoy/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for 

Grid scores on training set:
-1.016 (+/-0.882) for {'C': 1, 'gamma': 0.0001, 'kernel': 'rbf'}
-0.817 (+/-0.821) for {'C': 1, 'gamma': 0.001, 'kernel': 'rbf'}
-0.760 (+/-0.779) for {'C': 1, 'gamma': 0.01, 'kernel': 'rbf'}
-0.836 (+/-0.781) for {'C': 1, 'gamma': 0.1, 'kernel': 'rbf'}
-0.870 (+/-0.802) for {'C': 1, 'gamma': 0.2, 'kernel': 'rbf'}
-0.917 (+/-0.780) for {'C': 1, 'gamma': 0.5, 'kernel': 'rbf'}
-0.924 (+/-0.778) for {'C': 1, 'gamma': 0.6, 'kernel': 'rbf'}
-0.944 (+/-0.773) for {'C': 1, 'gamma': 0.9, 'kernel': 'rbf'}
-0.815 (+/-0.819) for {'C': 10, 'gamma': 0.0001, 'kernel': 'rbf'}
-0.759 (+/-0.787) for {'C': 10, 'gamma': 0.001, 'kernel': 'rbf'}
-0.781 (+/-0.786) for {'C': 10, 'gamma': 0.01, 'kernel': 'rbf'}
-0.959 (+/-0.815) for {'C': 10, 'gamma': 0.1, 'kernel': 'rbf'}
-1.008 (+/-0.806) for {'C': 10, 'gamma': 0.2, 'kernel': 'rbf'}
-1.158 (+/-0.694) for {'C': 10, 'gamma': 0.5, 'kernel': 'rbf'}
-1.185 (+/-0.684) for {'C': 10, 'gamma': 0.6, 'kernel': 'rbf'}
-1.161 (+/-0.681) for 

/Users/hridoy/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/Users/hridoy/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/Users/hridoy/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


-0.744 (+/-0.325) for {'C': 1, 'gamma': 0.01, 'kernel': 'rbf'}

2

In [80]:
import math
from math import sqrt

In [81]:
def svr_model(X, y):
    gsc = GridSearchCV(
        estimator=SVR(kernel='rbf'),
        param_grid={
            'C': [0.1, 1, 100, 1000],
            'epsilon': [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1, 5, 10],
            'gamma': [0.0001, 0.001, 0.005, 0.1, 1, 3, 5]
        },
        cv=5, scoring='neg_mean_squared_error', verbose=0, n_jobs=-1)

    grid_result = gsc.fit(X, y)
    best_params = grid_result.best_params_
    print(best_params)
    best_svr = SVR(kernel='rbf', C=best_params["C"], epsilon=best_params["epsilon"], gamma=best_params["gamma"],
                   coef0=0.1, shrinking=True,
                   tol=0.001, cache_size=200, verbose=False, max_iter=-1)

    scoring = {
               'abs_error': 'neg_mean_absolute_error',
               'squared_error': 'neg_mean_squared_error'}

    scores = cross_validate(best_svr, X, y, cv=10, scoring=scoring, return_train_score=True)
    
    return "MAE :", abs(scores['test_abs_error'].mean()), "| RMSE :", math.sqrt(abs(scores['test_squared_error'].mean()))
    

In [82]:
svr_model(X, y)

{'C': 100, 'epsilon': 0.5, 'gamma': 0.0001}


/Users/hridoy/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/Users/hridoy/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/Users/hridoy/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/Users/hridoy/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for 

('MAE :', 0.5654955386929288, '| RMSE :', 0.852204696960204)

MODELLING with best param

In [ ]:
{'C': 100, 'epsilon': 0.5, 'gamma': 0.0001}

In [83]:
clf = SVR(kernel='rbf', C=100, epsilon=0.5, gamma=0.0001,
                   coef0=0.1)
clf.fit(X_train, y_train)

y_pred = clf.predict( X_test)

/Users/hridoy/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


In [84]:
rmse = sqrt(mean_squared_error(y_pred, y_test))
print('Test RMSE: %.3f' % rmse)

Test RMSE: 0.804


In [85]:

# Scaling the predicted Price data back to original price scale
Predictions=TargetVarScalerFit.inverse_transform(y_pred)
 
# Scaling the y_test Price data back to original price scale
y_test_orig=TargetVarScalerFit.inverse_transform(y_test)
 
# Scaling the test data back to original scale
Test_Data=PredictorScalerFit.inverse_transform(X_test)

In [86]:
TestingData=pd.DataFrame(data=Test_Data, columns=Predictors)
TestingData['orginal']=y_test_orig
TestingData['Predicted']=Predictions
TestingData.head()

,Nox_ground,tavg,tmin,tmax,prcp,wdir,wspd,pres,orginal,Predicted
0,40.983786,3.2,0.1,5.6,0.5,89.0,11.3,1019.5,85.2440,132.421000
1,38.510068,14.2,9.0,18.6,0.0,268.0,19.7,1010.6,98.7010,112.205822
2,207.542204,11.3,7.8,15.1,0.0,77.0,9.7,1003.9,233.3780,219.769946
3,81.613131,11.1,6.4,13.1,1.0,208.0,23.5,1006.9,97.6385,112.506962
4,90.740024,8.2,4.2,10.4,8.4,160.0,17.1,979.0,83.1480,120.132263


In [94]:
# Computing the absolute percent error
APE=100*(abs(TestingData['orginal']-TestingData['Predicted'])/TestingData['orginal'])
TestingData['APE']=APE
 
print('The Accuracy of SVr model is:', 100-np.mean(APE))
TestingData.head()

The Accuracy of SVr model is: 62.030375830301494


,Nox_ground,tavg,tmin,tmax,prcp,wdir,wspd,pres,orginal,Predicted,APE
0,40.983786,3.2,0.1,5.6,0.5,89.0,11.3,1019.5,85.2440,132.421000,55.343485
1,38.510068,14.2,9.0,18.6,0.0,268.0,19.7,1010.6,98.7010,112.205822,13.682559
2,207.542204,11.3,7.8,15.1,0.0,77.0,9.7,1003.9,233.3780,219.769946,5.830907
3,81.613131,11.1,6.4,13.1,1.0,208.0,23.5,1006.9,97.6385,112.506962,15.228073
4,90.740024,8.2,4.2,10.4,8.4,160.0,17.1,979.0,83.1480,120.132263,44.480040


In [90]:
import numpy as np

def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [91]:
mean_absolute_percentage_error(TestingData['orginal'],TestingData['Predicted'])

37.969624169698506

In [93]:
from math import sqrt
rmse = sqrt(mean_squared_error(TestingData['Predicted'], TestingData['orginal']))
print('Test RMSE: %.3f' % rmse)

Test RMSE: 68.385
